<a href="https://colab.research.google.com/github/MUHAMMAD-IJLAL-061/Student-Study-Habits-Analysis/blob/main/Habits%26Activities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
project_path  = "/content/drive/MyDrive/ Study-Habits-and-Activities-of-Students/" # Corrected 'conten' to 'content'

import os
os.makedirs(project_path,exist_ok = True)
print(f"file created at {project_path}")

Mounted at /content/drive
file created at /content/drive/MyDrive/ Study-Habits-and-Activities-of-Students/


In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/afnansaifafnan/study-habits-and-activities-of-students")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: muhammadijlal061
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/afnansaifafnan/study-habits-and-activities-of-students


100%|██████████| 21.6k/21.6k [00:00<00:00, 62.7MB/s]

In [4]:
import pandas as pd
df = pd.read_csv("/content/study-habits-and-activities-of-students/student_lifestyle_dataset.csv")
# Save to Drive
df.to_csv(project_path + "Study-Habits-and-Activities-of-Students.csv", index=False)
print("✅ Study habbit_data.csv created successfully!")

✅ Study habbit_data.csv created successfully!


In [6]:
# Cleaning Data
df.dropna(inplace=True)
df = df[df["Study_Hours_Per_Day"] > 0]
df = df[df["Extracurricular_Hours_Per_Day"] > 0]
df = df[df["Physical_Activity_Hours_Per_Day"] > 0]
df = df[df["GPA"] > 0]

print("Data cleane ready for analytics")

Data cleane ready for analytics


In [8]:
lifestyle_cols = ['Study_Hours_Per_Day', 'Sleep_Hours_Per_Day', 'Physical_Activity_Hours_Per_Day', 'Social_Hours_Per_Day']
stress_analysis = df.groupby('Stress_Level')[lifestyle_cols].mean()
high_performers_df = df[df['GPA'] >= 3.5]
df_dropped = df.copy()
row_index_to_drop = 2
df_dropped.drop(labels=row_index_to_drop, axis=0, inplace=True)

print("Data grouped by sucessfull")

Data grouped by sucessfull


In [11]:
charts_path = os.path.join(project_path, "charts/")
os.makedirs(charts_path, exist_ok=True)

# Define colors for the plot
colors = {
    'Study_Hours_Per_Day': 'skyblue',
    'Sleep_Hours_Per_Day': 'lightcoral',
    'Physical_Activity_Hours_Per_Day': 'lightgreen',
    'Social_Hours_Per_Day': 'gold'
}

# Select only the relevant columns for this plot
columns_to_plot = ['Study_Hours_Per_Day', 'Sleep_Hours_Per_Day', 'Physical_Activity_Hours_Per_Day', 'Social_Hours_Per_Day']

# visualiazation 1
plt.figure(figsize=(12, 7))

# Plot the grouped data directly from the aggregated DataFrame, using only selected columns
stress_analysis[columns_to_plot].plot(
    kind='bar',
    rot=0,
    ax=plt.gca(), # Use current axes
    color=[colors[col] for col in columns_to_plot]
)

plt.title('Stress Level vs. Mean Daily Activity Hours', fontsize=16, weight='bold')
plt.ylabel('Average Hours Per Day')
plt.xlabel('Stress Level')

# Rename legend labels for clarity, matching the order of columns_to_plot
plt.legend(
    title='Activity',
    labels=['Study Hours', 'Sleep Hours', 'Physical Activity Hours', 'Social Hours'],
    frameon=True
)

plt.tight_layout()
plt.savefig(charts_path + "stress vs Daily Activity.png")
plt.close()

#viaualization 2

# --- 3. Chart Generation (Horizontal Bar Chart) ---
# Define time_allocation: Mean of lifestyle activities across all students
time_allocation = df[lifestyle_cols].mean()

plt.figure(figsize=(10, 6))

sns.barplot(
    x=time_allocation.values,
    y=time_allocation.index,
    hue=time_allocation.index, # Assign y to hue as suggested by FutureWarning
    palette='viridis',
    legend=False # Set legend to False as suggested by FutureWarning
)

plt.title('Overall Average Daily Time Allocation (All Students)', fontsize=16, weight='bold')
plt.xlabel('Average Hours Per Day')
plt.ylabel('Activity')
plt.tight_layout()


plt.savefig(charts_path + "Daily_Time_Allocation.png")
plt.close()

# Prepare data for the third plot: Impact of Study Hours on GPA and Stress Level
# Convert 'Stress_Level' to numerical values for plotting
stress_mapping = {'Low': 1, 'Moderate': 2, 'High': 3}
df['Stress_Numeric'] = df['Stress_Level'].map(stress_mapping)

# Group by Study_Hours_Per_Day and calculate mean GPA and Stress_Numeric
aggregated_data = df.groupby('Study_Hours_Per_Day')[['GPA', 'Stress_Numeric']].mean().reset_index()

# Sort for better plotting
aggregated_data = aggregated_data.sort_values(by='Study_Hours_Per_Day')

fig, ax1 = plt.subplots(figsize=(10, 6))

# --- AXIS 1 (Primary Y-axis: GPA) ---
color_gpa = '#3b82f6' # Swapped to Blue
ax1.set_xlabel('Study Hours Per Day', fontsize=12)
ax1.set_ylabel('Mean GPA', color=color_gpa, fontsize=12)

# Plot GPA line
line1 = ax1.plot(
    aggregated_data['Study_Hours_Per_Day'],
    aggregated_data['GPA'],
    color=color_gpa,
    label='Mean GPA',
    marker='o',
    markersize=8 # Increased marker size
)
ax1.tick_params(axis='y', labelcolor=color_gpa)
ax1.set_ylim(2.5, 4.0) # Set reasonable limits for GPA

# --- AXIS 2 (Secondary Y-axis: Stress Level) ---
# Create a secondary y-axis sharing the same x-axis
ax2 = ax1.twinx()
color_stress = '#ef4444' # Swapped to Red
ax2.set_ylabel('Mean Stress Level (1=Low, 3=High)', color=color_stress, fontsize=12)

# Plot Stress line
line2 = ax2.plot(
    aggregated_data['Study_Hours_Per_Day'],
    aggregated_data['Stress_Numeric'],
    color=color_stress,
    label='Mean Stress Level',
    linestyle='--', # Use a dashed line for differentiation
    marker='s',
    markersize=8 # Increased marker size
)
ax2.tick_params(axis='y', labelcolor=color_stress)
ax2.set_ylim(1.0, 3.0) # Set reasonable limits for Stress (1 to 3)

plt.title(
    'Impact of Study Hours on GPA and Stress Level',
    fontsize=16,
    weight='bold'
)

# Combine and display legends from both axes
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')

plt.grid(True)
plt.tight_layout()
plt.savefig(charts_path + "study_stress_GPA.png")
plt.close()

cleaned_path = project_path + "Cleaned_study_habits&Activities"
df.to_csv(cleaned_path, index=False)
print(f"✅ Cleaned data exported to: {cleaned_path}")

✅ Cleaned data exported to: /content/drive/MyDrive/ Study-Habits-and-Activities-of-Students/Cleaned_study_habits&Activities
